### Experimenting with Word2Vec to Predict Malicious Website
+ word2vec
+ sklearn
+ pandas
##### NB: Test Trial

In [1]:
# Load our packages
import gensim

In [2]:
# load EDA Packages
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("urldata.csv")

In [4]:
df.head()

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad
2,iamagameaddict.com,bad
3,kalantzis.net,bad
4,slightlyoffcenter.net,bad


In [5]:
df.url

0                                    diaryofagameaddict.com
1                                          espdesign.com.au
2                                        iamagameaddict.com
3                                             kalantzis.net
4                                     slightlyoffcenter.net
5                                          toddscarwash.com
6                                            tubemoviez.com
7                                                    ipl.hk
8               crackspider.us/toolbar/install.php?pack=exe
9                                           pos-kupang.com/
10                                               rupor.info
11        svision-online.de/mgfi/administrator/component...
12        officeon.ch.ma/office.js?google_ad_format=728x...
13                                              sn-gzzx.com
14                            sunlux.net/company/about.html
15                                              outporn.com
16                                   tim

In [7]:
Xw = df.url

In [ ]:
# let Xw be a list of tokenized texts (i.e. list of lists of tokens)
# model = gensim.models.Word2Vec(Xw, size=150)


In [8]:
 model = gensim.models.Word2Vec(
        Xw,
        size=150,
        window=10,
        min_count=2,
        workers=10)
   

In [9]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [10]:
# Functions for Vectorization and Word Embedding
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [11]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

In [13]:
Extree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
Extree_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

In [14]:
# Defining the Features(X),labels(y)
X = df.url
y = df.label

In [15]:
# Using Word2Vec With ExtraTreesClassifier to build a relationship between our features and labels
Extree_w2v.fit(X, y)

Pipeline(memory=None,
     steps=[('word2vec vectorizer', <__main__.MeanEmbeddingVectorizer object at 0x7f01938d1490>), ('extra trees', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impur...imators=200, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [16]:
# Predicting a website
test_X = [['kalantzis.net']]
print Extree_w2v.predict(test_X)

['good']


In [19]:
# Predicting a website
test_2 = [['google.com']]
print Extree_w2v.predict(test_2)

['good']


In [23]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Checking For Model Accuracy
Extree_w2v.score(X_test,y_test)
# First Score was 0.99

1.0